# Time Series predictions using LSTM

Yet another try

In [1]:
import pandas as pd
from pandas_datareader import data, wb
from datetime import datetime
import numpy as np
import graphviz
import random

import sys
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import bokeh.models
import bokeh.plotting as bk
bk.output_notebook()

import ts_helpers

Loading BokehJS ...

In [2]:
np.random.seed(42)
random.seed(42)

WINDOW_IN = 7
WINDOW_OUT = 4

TICKERS = [pd.read_csv('tickers/%s' % f)
           for f in os.listdir('tickers')
           if f.endswith('.csv')
          ]
TICKERS = [ts_helpers.build_corpus(
                    df,
                    wnd_in=WINDOW_IN,
                    wnd_out=WINDOW_OUT)
            for df in TICKERS
            if df.shape[0] >= 364
]

In [3]:
class RNN(nn.Module): 
    
    def __init__(self, n_in, n_mem, days_out):
        super(RNN, self).__init__()
        self.n_in, self.n_mem, self.days_out = n_in, n_mem, days_out
        self.xm2f = nn.Linear(in_features=(n_mem + n_in), out_features=n_mem)
        self.xm2invent = nn.Linear(in_features=(n_mem + n_in), out_features=n_mem)
        self.xm2save = nn.Linear(in_features=(n_in + n_mem), out_features=n_mem)
        self.x2mid = nn.Linear(in_features=n_in, out_features=n_mem)
        self.mid2o = nn.Linear(in_features=n_mem, out_features=days_out)
        
    def init_weights(self, var=.1):
        for tr in [
                self.xm2f,
                self.xm2invent,
                self.xm2save,
                self.x2mid,
                self.mid2o,
                ]:
            tr.weight.data.uniform_(-var, var)
    
    def forward(self, x, mem):
        xmem = torch.cat((x, mem))
        mem = mem * F.sigmoid(self.xm2f(xmem))
        new_mem = F.tanh(self.xm2invent(xmem))
        mem_mask = F.sigmoid(self.xm2save(xmem))
        mem = ((1 - mem_mask) * mem
               + mem_mask * new_mem
        )
        mid = self.x2mid(x) * mem
        o = self.mid2o(mid)
        sgn = torch.stack((F.sigmoid(-o), F.sigmoid(o)), dim=1)
        return (o, sgn, mem)
    
    def init_mem(self):
        return Variable(torch.zeros(self.n_mem))

In [4]:
OBJECTIVE = ts_helpers.my_objective

In [5]:
rnn = RNN(n_in=TICKERS[0][0].shape[1], n_mem=64, days_out=TICKERS[0][1].shape[1])
rnn.init_weights(.1)
optimizer = torch.optim.RMSprop(rnn.parameters(), lr=.001)

In [6]:
from sklearn.model_selection import train_test_split
TR, TE = train_test_split(TICKERS)

In [7]:
def get_tr():
    return random.choice(TR)

def get_te():
    return random.choice(TE)

In [8]:
def train(model, epochs=2):
    for epoch in range(1, epochs+1):
        if epoch % 8 == 1:
            X, y = get_tr()
        print(
            'Epoch #%s; Loss: %s'
            % (
                epoch,
                ts_helpers.loss(model, X, y,
                     OBJECTIVE,
                     optimizer)))

In [9]:
train(rnn, epochs=64)

Epoch #1; Loss: 0.203264057636261
Epoch #2; Loss: 0.19492176175117493
Epoch #3; Loss: 0.18805454671382904
Epoch #4; Loss: 0.1803928017616272
Epoch #5; Loss: 0.17362312972545624
Epoch #6; Loss: 0.16876421868801117
Epoch #7; Loss: 0.16382946074008942
Epoch #8; Loss: 0.15842978656291962
Epoch #9; Loss: 0.039977263659238815
Epoch #10; Loss: 0.034471213817596436
Epoch #11; Loss: 0.03342147544026375
Epoch #12; Loss: 0.03292109817266464
Epoch #13; Loss: 0.032563865184783936
Epoch #14; Loss: 0.032267238944768906
Epoch #15; Loss: 0.032002393156290054
Epoch #16; Loss: 0.031760409474372864
Epoch #17; Loss: 0.3877580165863037
Epoch #18; Loss: 0.36414629220962524
Epoch #19; Loss: 0.3605678379535675
Epoch #20; Loss: 0.35775992274284363
Epoch #21; Loss: 0.355023592710495
Epoch #22; Loss: 0.3521744906902313
Epoch #23; Loss: 0.34916192293167114
Epoch #24; Loss: 0.3459714651107788
Epoch #25; Loss: 0.9184245467185974
Epoch #26; Loss: 0.8703881502151489
Epoch #27; Loss: 0.8571823835372925
Epoch #28; Loss:

In [10]:
Xte, yte = get_te()
ts_helpers.evaluate_model(rnn, Xte, yte,
               include_plohi=False,
               title='Evaluation on random test set')

In [11]:
train(rnn, epochs=500)

Epoch #1; Loss: 0.4233331084251404
Epoch #2; Loss: 0.40003877878189087
Epoch #3; Loss: 0.3923546373844147
Epoch #4; Loss: 0.38659846782684326
Epoch #5; Loss: 0.38095682859420776
Epoch #6; Loss: 0.37503814697265625
Epoch #7; Loss: 0.3692554831504822
Epoch #8; Loss: 0.38187524676322937
Epoch #9; Loss: 1.2704195976257324
Epoch #10; Loss: 1.195749282836914
Epoch #11; Loss: 1.1688570976257324
Epoch #12; Loss: 1.150814175605774
Epoch #13; Loss: 1.1360541582107544
Epoch #14; Loss: 1.1224697828292847
Epoch #15; Loss: 1.109032392501831
Epoch #16; Loss: 1.0951701402664185
Epoch #17; Loss: 0.09807979315519333
Epoch #18; Loss: 0.09156191349029541
Epoch #19; Loss: 0.09003223478794098
Epoch #20; Loss: 0.08933458477258682
Epoch #21; Loss: 0.08890461176633835
Epoch #22; Loss: 0.08858007937669754
Epoch #23; Loss: 0.08830514550209045
Epoch #24; Loss: 0.08805512636899948
Epoch #25; Loss: 0.06028704717755318
Epoch #26; Loss: 0.05792377516627312
Epoch #27; Loss: 0.056985046714544296
Epoch #28; Loss: 0.0558

KeyboardInterrupt: 

In [12]:
Xte, yte = get_te()
ts_helpers.evaluate_model(rnn, Xte, yte,
               include_plohi=True, include_tlohi=True,
               title='Evaluation on the test set')

In [13]:
Xte, yte = get_te()
ts_helpers.evaluate_model(rnn, Xte, yte,
               include_tlohi=True,
               title='Evaluation on a random test set')

In [14]:
Xte, yte = get_te()
ts_helpers.evaluate_model(rnn, Xte, yte,
               include_tlohi=True,
               title='Evaluation on a test set')

In [15]:
%%javascript
IPython.notebook.kernel.execute('nb_name = ' + '"' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [16]:
torch.save(rnn.state_dict(), nb_name + '.weights')

In [17]:
Xte, yte = get_te()
ts_helpers.evaluate_model(rnn, Xte, yte,
               include_tlohi=True,
               title='Evaluation on a random test set')